In [1]:
# Decision Tree Classifier
# Removed Empty Data Records

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

# Replace '?' with NaN
merged_dataset.replace('?', np.nan, inplace=True)

# Convert all columns to numeric
merged_dataset = merged_dataset.apply(pd.to_numeric, errors='coerce')

# Drop rows where the target variable is NaN
merged_dataset.dropna(subset=['target'], inplace=True)  # Replace 'target_column_name' with your actual target column name

# Define features and target variable
X = merged_dataset.drop('target', axis=1)  # Replace 'target_column_name' with your actual target column name
y = merged_dataset['target']  # Replace 'target_column_name' with your actual target column name

# Impute missing values in features using the mean of each column
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree classifier
model = DecisionTreeClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Model Accuracy: {accuracy * 100:.2f}%")


ModuleNotFoundError: No module named 'sklearn'